DATASETS

In [35]:
!pip install -U beir
!pip install nltk
!pip install pandas

In [36]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from beir import util

In [37]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
dataset_url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
def load_dataset(dataset_name):
    out_dir = os.path.join(os.getcwd(), 'datasets')
    util.download_and_unzip(dataset_url.format(dataset_name), out_dir)

In [39]:
def filter_words(w, filter_stopwords):
    if filter_stopwords:
        return w.isalnum() and w not in stop_words
    else:
        return w

In [40]:
def build_doc_stats(dataset_name, fdist, doc_count, sentence_count, query_count):
    term_count = fdist.N()
    bin_count = fdist.B()
    word_per_doc = term_count / doc_count
    valid_sentence_count = sentence_count
    if valid_sentence_count == 0:
        print('WARNING! "{}" corpus does not have title values.'.format(dataset_name))
        valid_sentence_count = 1
    word_per_sent = term_count / valid_sentence_count
    stats_dict = {
        '# of docs': [doc_count],
        '# of terms': [term_count],
        '# of unique terms': [bin_count],
        '# of sentences': [sentence_count],
        '# of terms per sentences': [word_per_sent],
        '# of terms per docs': [word_per_doc]}
    if query_count != 0:
        stats_dict['# of queries'] = query_count
    stats_df = pd.DataFrame(data=stats_dict)
    stats_df.index = [dataset_name]
    return stats_df

In [41]:
def get_corpus_df(dataset_name, file_name):
    corpus_path = os.path.join(os.getcwd(), 'datasets', dataset_name, file_name)
    return pd.read_json(path_or_buf=corpus_path, lines=True)

In [42]:
def stats_corpus(dataset_name, column_name, add_queries):
    load_dataset(dataset_name)
    query_count = 0
    if add_queries:
        queries_df = get_corpus_df(dataset_name, 'queries.jsonl')
        query_count = len(queries_df.index)
    corpus_df = get_corpus_df(dataset_name, 'corpus.jsonl')
    doc_count = len(corpus_df.index)
    sentence_count = 0
    fdist = FreqDist()
    for line in corpus_df.get(column_name):
        sentence_count += len(sent_tokenize(line))
        tokens = [w.lower() for w in word_tokenize(line) if filter_words(w, True)]
        for word in tokens:
            fdist[word] += 1
    return build_doc_stats(dataset_name, fdist, doc_count, sentence_count, query_count)

In [43]:
datasets = ['arguana', 'fiqa']
title_stats = []
text_stats = []
for dataset in datasets:
    title_stats.append(stats_corpus(dataset, 'title', False))
for dataset in datasets:
    text_stats.append(stats_corpus(dataset, 'text', True))

WARNING! "fiqa" corpus does not have title values.




---



**ARGUANA** - [Homepage](http://argumentation.bplaced.net/arguana/data) - [Paper](https://aclanthology.org/P18-1023.pdf)

**FIQA** - [Homepage](https://sites.google.com/view/fiqa/home) - [Paper](https://www.researchgate.net/publication/324629350_WWW'18_Open_Challenge_Financial_Opinion_Mining_and_Question_Answering)



---



**STATISTICS FOR TITLES**

In [44]:
pd.concat(title_stats)

,# of docs,# of terms,# of unique terms,# of sentences,# of terms per sentences,# of terms per docs
arguana,8674,24362,501,2699,9.026306,2.808623
fiqa,57638,0,0,0,0.000000,0.000000


**STATISTICS FOR DOCUMENTS**

In [45]:
pd.concat(text_stats)

,# of docs,# of terms,# of unique terms,# of sentences,# of terms per sentences,# of terms per docs,# of queries
arguana,8674,822246,33063,65730,12.509448,94.794328,1406
fiqa,57638,4187377,66790,419639,9.978522,72.649589,6648


end of fun.